In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#!pip install fanalysis

from sklearn.metrics import confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
# les noms des variables

colonnes =(["IDENT", "TypeHabi", "GeoFaz_Scl", "GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Anciennete",
            "Logement_prix_m2", "Mediane_RFR", "Sexe", "Prenom_Phase_scl_H", "Prenom_Phase_scl_F",
            "Prenom_Phase_cl_H", "Prenom_Phase_cl_F", "Age_H", "Age_F", "Affinite_1", "Affinite_2",
            "Prenom_effectif", "Potentiel_prenom", "Onoma-Phase_Scl", "Onoma-Phase_CL", "Top_mon"])

In [30]:
colonnes_montant = ["DEDUPLI", "IDENT", "SommeDeMONTANT_NUM", "Flag_Top_Montant", "NovosScore_Vingtile"]

data_montant_orig = pd.read_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/DONS_FDF_N_TODAY_1ers_gestes_REF_MD.txt", sep=";", header = 0, low_memory=False, dtype=str) #, dtype=str )#, dtype='Int64')

data_montant_orig.iloc[:,2]=data_montant_orig.iloc[:,2].str.replace(",",".")

data_montant_orig.columns =colonnes_montant
data_montant_orig["SommeDeMONTANT_NUM"]=data_montant_orig["SommeDeMONTANT_NUM"].astype(float)

In [31]:
# importation du fichier (changer le chemin d'accès)
''' Faites attention au type de fichier, au délimiteur des valeurs (ça doit être unique et non multiple),
au header et à la lecture des types d'objets (low_memory) '''

data_orig = pd.read_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/sans_8_new_my_data_FDFTOT21.txt", delim_whitespace="\t", header = None, names=colonnes, low_memory=False,dtype=str )#, dtype='Int64')


In [32]:
''' copie superficielle (peu profonde) du dataset original pour ne pas modifier les valeurs
d'origine par erreur plus tard 
'''
data = data_orig.copy()
data.shape

(1192180, 23)

In [33]:
# visualisation des premiers lignes
data.head()

,IDENT,TypeHabi,GeoFaz_Scl,GeoFaz_cl,ImmoFaz_Scl,ImmoFaz_cl,Anciennete,Logement_prix_m2,Mediane_RFR,Sexe,Prenom_Phase_scl_H,Prenom_Phase_scl_F,Prenom_Phase_cl_H,Prenom_Phase_cl_F,Age_H,Age_F,Affinite_1,Affinite_2,Prenom_effectif,Potentiel_prenom,Onoma-Phase_Scl,Onoma-Phase_CL,Top_mon
0,0010000001,02,81,08,91,09,06,21,21,01,71,41,07,04,12,04,20,02,02,01,51,05,1
1,0010000006,01,75,07,43,04,06,06,13,02,71,64,07,06,12,09,04,12,04,13,12,01,1
2,0010000011,01,76,07,43,04,06,06,16,03,54,81,05,08,07,12,06,06,04,18,36,03,2
3,0010000013,03,75,07,43,04,04,06,13,01,71,36,07,03,12,11,11,19,04,06,21,02,1
4,0010000015,02,33,03,44,04,06,07,12,01,71,42,07,04,12,05,14,03,03,01,14,01,2


In [34]:
#nombre de modalités de chaque variables (dans un dataframe)
def nombre_mod(data2):
    #création d'un dataframe vide
    dff= pd.DataFrame(columns=["VARIABLE","NOMBRE_MOD", "NOMBRE_MOD_SANS_NA"])
    for i in range(data2.shape[1]): # sans compter les valeurs manquantes
        taille =len(data2.iloc[:,i].value_counts(dropna=False))
        taille_1 = len(data2.iloc[:,i].value_counts(dropna=True) )
        dff=dff.append({"VARIABLE": colonnes[i],"NOMBRE_MOD":taille, "NOMBRE_MOD_SANS_NA":taille_1 }, ignore_index=True)
    return dff

# sélection des corrélations les plus importantes
def variables_corr(tableau_corr, seuil):
    colonnes = tableau_corr.columns
    val_temp = 0 
    variables_corr_pos=[] ; variables_corr_pos_val=[]
    variables_corr_neg=[] ; variables_corr_neg_val=[]
    for i in range(tableau_corr.shape[0]):  
        for j in range(tableau_corr.shape[0]):
            if i<j: #i!=j:
                if (tableau_corr.iloc[i,j] > seuil):
                    variables_corr_pos.append(colonnes[j])
                    variables_corr_pos_val.append(round( tableau_corr.iloc[i,j], 2))
                    val_temp += 1

                if (tableau_corr.iloc[i,j] <-seuil):
                    variables_corr_neg.append(colonnes[j])
                    variables_corr_neg_val.append(round( tableau_corr.iloc[i,j], 2))
                    val_temp += 1

        if (len(variables_corr_pos) > 0) | (len(variables_corr_neg) >0) :
            print("variable : ", colonnes[i])
            if len(variables_corr_pos) > 0:    
                print("corr pos", variables_corr_pos)
                print("corr pos", variables_corr_pos_val)
                val_temp += 1

            if len(variables_corr_neg) >0:  
                print("corr neg", variables_corr_neg)
                print("corr neg", variables_corr_neg_val)
                val_temp += 1
            print("")
        variables_corr_pos = variables_corr_pos*0
        variables_corr_neg = variables_corr_pos*0

        variables_corr_pos_val = variables_corr_pos_val*0
        variables_corr_neg_val = variables_corr_pos_val*0
    if val_temp == 0:
        print("IL N'Y A PAS DE CORRELATION ENTRE LES VARIABLES PAR RAPPORT AU SEUIL PRIS")
        

def evaluation(model, X_train, y_train, X_test, y_test, scoring, plot):

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("test set : ")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred)) 
    print("train set : ")
    print(confusion_matrix(y_train, model.predict(X_train)))  
    print(classification_report(y_train, model.predict(X_train) ))

    if plot == True:
        N, train_score, val_score = learning_curve(model, X_train, y_train, shuffle=True, cv = 4, scoring = scoring, train_sizes = np.linspace(0.1, 1, 6) )

        plt.figure(figsize = (10,6))
        plt.plot(N, train_score.mean(axis=1), label ="train_score")
        plt.plot(N, val_score.mean(axis=1), label ="cross_validation_score")
        plt.legend()
    else :
        print("GRAPHIQUE NON DEMANDE")

# Comme il n'y a pas de valeurs manquantes (à la place il y a un nombre), le seul traitement qu'on fait c'est tranformer
# le type de variable (les variables continues deviennent qualitatives)
def data_processing(df):
    """ 
    for i in range(0,len(df.columns)):    
        df[df.columns[i]].fillna(100.0, inplace=True) #On remplace les données manquantes par 100
    for i in range(0,len(df.columns)):
        print(df[df.columns[i]].value_counts(dropna=False))    
    """
    
   #Transformation de toutes les variables en facteurs         
    for i in range(0,len(df.columns)):
        df[df.columns[i]] = df[df.columns[i]].astype('category')
    return df.to_numpy() #.as_matrix()


def vingtiles(model, data_orig, affichage, save, path): # data_orig est une variables locale (peut être différente au df d'origine)
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe Top_mon)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var])
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # vingtiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    #print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    #print(df_tous)

    # création de la dernière colonne CLASSE et son remplissage
    index_20_ord = pd.DataFrame(np.unique(df_tous["TRANCHES"])) 
    df_tous["CLASSE"]=df_tous["TRANCHES"].copy()
    ind_val=1
    for kk in range(20):
        df_tous["CLASSE"][df_tous["CLASSE"]== index_20_ord.iloc[kk,0] ] = str(ind_val) 
        ind_val+=1
    print(df_tous)    
    
    # pour la colonne Top_mon : changement de type de variable et remplissage des valeurs (s'il le faut)
    data_temp_t = data_orig.copy()
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(1.0,'1')
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(2.0,'2')
    data_all_complet_class = data_temp_t #data_temp_t.fillna('2')
    #data_all_complet_class

    # jointure
    ident_class_tous = data_all_complet_class[["IDENT","Top_mon"]]
    tableau_resultats = pd.merge(df_tous,ident_class_tous)
    #tableau_resultats

    # tranches finales
    tab_cross_temp = pd.crosstab(tableau_resultats["TRANCHES"],tableau_resultats["Top_mon"])
    #print(tab_cross_temp); print("-----------------------")

    # création d'une colonne TOTAL
    tab_cross_temp["TOTAL"]=tab_cross_temp["1"]+tab_cross_temp["2"]
    
    # affichage
    if affichage ==True:        
        print(tab_cross_temp)

    # création du tableau final en éliminant la colonne 2
    #tab_pour_excel = tab_cross_temp.drop(["2"], axis=1)

    # si on ne veut pas éliminer la colonne 2
    tab_pour_excel =  tab_cross_temp
    
    # enregistrement sous Excel
    if save == True:
        tab_pour_excel.to_excel(path)

    return  df_tous


def tableau_5_montant(df_tous, data_montant_orig, affichage, save, path):
    # tableau avec les deux variables qui nous intéressent
    ident_et_montant = data_montant_orig[["IDENT","SommeDeMONTANT_NUM"]]

    # jointure
    tableau_resultats_et_montant = pd.merge(df_tous,ident_et_montant)
    #print(tableau_resultats_et_montant) 

    # on élimine toutes les sommes supérieur à 5000 €
    tableau_resultats_et_montant = tableau_resultats_et_montant[tableau_resultats_et_montant["SommeDeMONTANT_NUM"] <= 5000]
    print(tableau_resultats_et_montant) 
    #print(tableau_resultats_et_montant["TRANCHES"].value_counts())
    #print(pd.DataFrame(np.unique(tableau_resultats_et_montant["TRANCHES"])))

    # index
    index_temp = pd.DataFrame(np.unique(tableau_resultats_et_montant["TRANCHES"])) 

    # création du tableau qui contiendra les résultats finaux
    tableau_resultats_finaux = pd.DataFrame(columns=["TRANCHES","Moyen_Somme","TOTAL"],index=range(20))
    tableau_resultats_finaux["TRANCHES"] = index_temp

    # remplissage du tableau final
    for k in range(20):
        tableau_resultats_finaux.iloc[k,1] = tableau_resultats_et_montant[tableau_resultats_et_montant["TRANCHES"]==index_temp.iloc[k,0]]["SommeDeMONTANT_NUM"].mean()
        tableau_resultats_finaux.iloc[k,2] = (tableau_resultats_et_montant["TRANCHES"]==index_temp.iloc[k,0]).sum()
    
    # affichage
    if affichage:        
        print(tableau_resultats_finaux)
    
    # enregistrement sous Excel
    if save:
        tableau_resultats_finaux.to_excel(path)


def vingtiles_ind_sans_target(model, data_orig, affichage, save, path):
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe IFI+)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var])
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # quarantiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    print(df_tous)
    print(df_tous["TRANCHES"].value_counts(ascending = True))

    # GERE L'ORDRE
    patra = np.unique(df_tous.iloc[:,2], return_counts=True)
    df_tranches_sans_target = pd.DataFrame([patra[0],patra[1]]).T
    df_tranches_sans_target.columns=["TRANCHES","COUNTS"]

    # affichage
    if affichage ==True:        
        print(df_tranches_sans_target)
    
    # enregistrement sous Excel
    if save == True:
        df_tranches_sans_target.to_excel(path)
    

# élimination des variables trop corrélées (choix manuel)
def elimination_val_corr(data):
    data = data.drop(columns=[""])
    return data
          


In [35]:
def vingtiles_xgb(model, data_orig, affichage, save, path):
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe IFI+)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var].values)
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # quarantiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    #print(df_tous)
    
    # pour la colonne Top_mon :
    # remplissage de la dernière colonne par 2 (VALABLE QUE POUR CE DATA, MODIFIER POUR D'AUTRES)
    data_temp_t = data_orig.copy()
    # remplacement
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(1.0,'1')
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(2.0,'2')
    data_all_complet_class = data_temp_t.fillna('2')
    #data_all_complet_class

    # jointure
    ident_class_tous = data_all_complet_class[["IDENT","Top_mon"]]
    tableau_resultats = pd.merge(df_tous,ident_class_tous)
    #tableau_resultats

    # tranches finales
    tab_cross_temp = pd.crosstab(tableau_resultats["TRANCHES"],tableau_resultats["Top_mon"])
    #print(tab_cross_temp); print("-----------------------")

    # création d'une colonne total
    tab_cross_temp["TOTAL"]=tab_cross_temp["1"]+tab_cross_temp["2"]
    
    # affichage
    if affichage ==True:        
        print(tab_cross_temp)

    # création du tableau final en éliminant la colonne 2
    #tab_pour_excel = tab_cross_temp.drop(["2"], axis=1)

    # si on ne veut pas éliminer la colonne 2
    tab_pour_excel =  tab_cross_temp
    
    # enregistrement sous Excel
    if save == True:
        tab_pour_excel.to_excel(path)

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier

from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve, GridSearchCV

In [37]:
####################  DONNEES SANS LES VARIABLES CORR ET ENCODAGE DUMMIES        ###########################

In [38]:
# test predict individus sans classe
data = data_orig.copy()

# on élimine les variables dons
data=data.drop(["GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Prenom_Phase_cl_H", "Prenom_Phase_cl_F","Onoma-Phase_Scl",
                "Onoma-Phase_CL"],axis=1)

# encodage Onehot sur toutes les données (on évite ainsi les erreurs futures)
nb_var = data.shape[1] - 1
data_1_22 = pd.get_dummies(data.iloc[:,1:(data.shape[1]-1)], columns=data.columns[1:(data.shape[1]-1)] )
data_var_qualitatives_temp =  pd.concat([data["IDENT"], data_1_22, data["Top_mon"]], axis=1)
data = data_var_qualitatives_temp
#encodage = OneHotEncoder()
#encodage.fit(data_processing(data.iloc[:,1:nb_var]))

# élimination des individus n'ayant pas de valeur pour la varialbe IFI+
"""IMPORTANT"""
data_sans_nan = data[(data.isnull()).sum(axis=1) == 0]

#conversion du type de variable
for i in range(1,data_sans_nan.shape[1]):
    data_sans_nan.iloc[:,i]=data_sans_nan.iloc[:,i].astype(int)

# base de données avec des variables corrélées et avec encodage OneHot
data =data_sans_nan

nb_var = data.shape[1] - 1
train_set, test_set = train_test_split(data, test_size = 0.2, random_state=0 )

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [12]:
####################################################################################
#                            MODELE 1 : BOOSTING
####################################################################################

In [13]:
model_AdaBoost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced"), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__max_depth' : list(range(2,20, 2)), 'n_estimators' : [20, 50] }

grid_AdaBoost = GridSearchCV(model_AdaBoost, param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_AdaBoost = grid_AdaBoost.fit(train_X_s, train_Y_s)
resultat_grid_AdaBoost.best_params_
#model_AdaBoost.get_params


{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'n_estimators': 20}

In [39]:
final_model_AdaBoost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(class_weight="balanced", criterion="gini",max_depth=2),n_estimators = 20 , random_state=0 ) 
evaluation(final_model_AdaBoost, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall" , plot =False)

test set : 
[[2826 2505]
 [4948 9581]]
              precision    recall  f1-score   support

           1       0.36      0.53      0.43      5331
           2       0.79      0.66      0.72     14529

    accuracy                           0.62     19860
   macro avg       0.58      0.59      0.58     19860
weighted avg       0.68      0.62      0.64     19860

train set : 
[[11185 10212]
 [19170 38869]]
              precision    recall  f1-score   support

           1       0.37      0.52      0.43     21397
           2       0.79      0.67      0.73     58039

    accuracy                           0.63     79436
   macro avg       0.58      0.60      0.58     79436
weighted avg       0.68      0.63      0.65     79436

GRAPHIQUE NON DEMANDE


In [40]:
# TRAIN SET TEST SET
vingtiles_tableau = vingtiles(model = final_model_AdaBoost, data_orig = data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Adaboost.xlsx")

            IDENT     PROBA
63493  0010397061  0.135818
18843  0010291953  0.136391
10385  0010257272  0.136550
62181  0010395560  0.136550
11441  0010258353  0.136550
...           ...       ...
99283  0010539458  0.532344
88492  0010524610  0.532355
83980  0010505638  0.532984
79062  0010415013  0.533166
93101  0010530839  0.534106

[99296 rows x 2 columns]
0.499496    4341
0.496758    3719
0.491285    2614
0.493052    2507
0.501509    1123
            ... 
0.506313       1
0.518211       1
0.495084       1
0.499973       1
0.497490       1
Name: PROBA, Length: 8264, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            IDENT     PROBA                       TRANCHES CLASSE
63493  0010397061  0.135818  (0.13570000000000002, 0.4908]      1
18843  0010291953  0.136391  (0.13570000000000002, 0.4908]      1
10385  0010257272  0.136550  (0.13570000000000002, 0.4908]      1
62181  0010395560  0.136550  (0.13570000000000002, 0.4908]      1
11441  0010258353  0.136550  (0.13570000000000002, 0.4908]      1
...           ...       ...                            ...    ...
99283  0010539458  0.532344               (0.5119, 0.5341]     20
88492  0010524610  0.532355               (0.5119, 0.5341]     20
83980  0010505638  0.532984               (0.5119, 0.5341]     20
79062  0010415013  0.533166               (0.5119, 0.5341]     20
93101  0010530839  0.534106               (0.5119, 0.5341]     20

[99296 rows x 4 columns]
Top_mon                           1     2  TOTAL
TRANCHES                                        
(0.13570000000000002, 0.4908]   656  4331   4987
(0.4908, 0.4919]                788

In [41]:
# TABLEAU MONTANT
tableau_5_montant(vingtiles_tableau, data_montant_orig, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/montant_Adaboost.xlsx")

            IDENT     PROBA  ... CLASSE SommeDeMONTANT_NUM
0      0010397061  0.135818  ...      1               15.0
1      0010291953  0.136391  ...      1               30.0
2      0010257272  0.136550  ...      1               10.0
3      0010395560  0.136550  ...      1               15.0
4      0010258353  0.136550  ...      1               15.0
...           ...       ...  ...    ...                ...
99291  0010539458  0.532344  ...     20             1917.6
99292  0010524610  0.532355  ...     20               20.0
99293  0010505638  0.532984  ...     20              100.0
99294  0010415013  0.533166  ...     20              100.0
99295  0010530839  0.534106  ...     20              200.0

[99146 rows x 5 columns]
                         TRANCHES Moyen_Somme TOTAL
0   (0.13570000000000002, 0.4908]     28.2102  4985
1                (0.4908, 0.4919]     29.7559  4944
2                (0.4919, 0.4931]     30.9221  4965
3                (0.4931, 0.4939]     31.4666  5183
4     

In [42]:
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_AdaBoost, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/ind_sans_target_Adaboost.xlsx")

              IDENT     PROBA
201285   0050041980  0.134791
640373   0250013157  0.135341
756347   0280069125  0.135678
1126718  0370043886  0.135818
867315   0320008483  0.136028
...             ...       ...
636629   0250009413  0.538215
287139   0110005118  0.538356
895764   0340001932  0.538955
636562   0250009346  0.538955
628755   0250001539  0.539339

[1092884 rows x 2 columns]
0.499496    50475
0.491285    34483
0.493052    31305
0.496758    17272
0.501509    16702
            ...  
0.495653        1
0.494264        1
0.493699        1
0.501168        1
0.492433        1
Name: PROBA, Length: 22106, dtype: int64
                                 PROBA
201285   (0.13470000000000001, 0.4908]
640373   (0.13470000000000001, 0.4908]
756347   (0.13470000000000001, 0.4908]
1126718  (0.13470000000000001, 0.4908]
867315   (0.13470000000000001, 0.4908]
...                                ...
636629                (0.5142, 0.5393]
287139                (0.5142, 0.5393]
895764                

In [ ]:
#############################################################
#                   MODELE 2 : RANDOM FOREST
#############################################################

In [14]:
model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'], 'max_depth' : [2,3,4,5],"max_features":[8,12,15,20,25,30] ,'n_estimators' : [20,50,100,200] }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s, train_Y_s)
resultat_grid_RandomForestClassifier.best_params_


{'criterion': 'gini', 'max_depth': 3, 'max_features': 25, 'n_estimators': 100}

In [43]:
final_model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", criterion="gini", max_depth=3, max_features=25, n_estimators = 100, random_state=0) 
evaluation(final_model_RandomForestClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[3380 1951]
 [6795 7734]]
              precision    recall  f1-score   support

           1       0.33      0.63      0.44      5331
           2       0.80      0.53      0.64     14529

    accuracy                           0.56     19860
   macro avg       0.57      0.58      0.54     19860
weighted avg       0.67      0.56      0.58     19860

train set : 
[[13273  8124]
 [27007 31032]]
              precision    recall  f1-score   support

           1       0.33      0.62      0.43     21397
           2       0.79      0.53      0.64     58039

    accuracy                           0.56     79436
   macro avg       0.56      0.58      0.53     79436
weighted avg       0.67      0.56      0.58     79436

GRAPHIQUE NON DEMANDE


In [44]:
# TRAIN SET TEST SET
vingtiles_tableau = vingtiles(model = final_model_RandomForestClassifier, data_orig = data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Random_Forest.xlsx")

            IDENT     PROBA
67904  0010401663  0.407182
53738  0010379725  0.408820
21093  0010337388  0.408885
42914  0010368050  0.409773
93208  0010530978  0.410760
...           ...       ...
88310  0010524335  0.694018
51004  0010376561  0.694018
3728   0010049725  0.694018
54667  0010381112  0.694018
2071   0010027021  0.694018

[99296 rows x 2 columns]
0.449338    1855
0.526649    1347
0.451151     695
0.510004     676
0.461725     508
            ... 
0.510846       1
0.434282       1
0.499693       1
0.444811       1
0.462459       1
Name: PROBA, Length: 31410, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            IDENT     PROBA                     TRANCHES CLASSE
67904  0010401663  0.407182  (0.40599999999999997, 0.43]      1
53738  0010379725  0.408820  (0.40599999999999997, 0.43]      1
21093  0010337388  0.408885  (0.40599999999999997, 0.43]      1
42914  0010368050  0.409773  (0.40599999999999997, 0.43]      1
93208  0010530978  0.410760  (0.40599999999999997, 0.43]      1
...           ...       ...                          ...    ...
88310  0010524335  0.694018               (0.585, 0.694]     20
51004  0010376561  0.694018               (0.585, 0.694]     20
3728   0010049725  0.694018               (0.585, 0.694]     20
54667  0010381112  0.694018               (0.585, 0.694]     20
2071   0010027021  0.694018               (0.585, 0.694]     20

[99296 rows x 4 columns]
Top_mon                         1     2  TOTAL
TRANCHES                                      
(0.40599999999999997, 0.43]   733  4238   4971
(0.43, 0.435]                 871  4121   4992
(0.435, 0.445]    

In [45]:
# TABLEAU MONTANT
tableau_5_montant(vingtiles_tableau, data_montant_orig, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/montant_Random_Forest.xlsx")

            IDENT     PROBA  ... CLASSE SommeDeMONTANT_NUM
0      0010401663  0.407182  ...      1               50.0
1      0010379725  0.408820  ...      1                5.0
2      0010337388  0.408885  ...      1               15.0
3      0010368050  0.409773  ...      1               20.0
4      0010530978  0.410760  ...      1               15.0
...           ...       ...  ...    ...                ...
99291  0010524335  0.694018  ...     20               50.0
99292  0010376561  0.694018  ...     20              500.0
99293  0010049725  0.694018  ...     20                5.0
99294  0010381112  0.694018  ...     20              500.0
99295  0010027021  0.694018  ...     20               50.0

[99146 rows x 5 columns]
                       TRANCHES Moyen_Somme TOTAL
0   (0.40599999999999997, 0.43]      28.702  4970
1                 (0.43, 0.435]     31.2005  4991
2                (0.435, 0.445]     31.4399  4941
3                (0.445, 0.449]     32.9147  5764
4               

In [46]:
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_RandomForestClassifier, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/ind_sans_target_Random_Forest.xlsx")

              IDENT     PROBA
1044351  0360121519  0.407182
1117635  0370034803  0.407182
620301   0240003092  0.407692
554085   0210011337  0.407692
299901   0110017880  0.407692
...             ...       ...
407129   0130055113  0.694018
1153801  0370070969  0.694018
383445   0130031429  0.694018
691558   0280004336  0.694018
895764   0340001932  0.697864

[1092884 rows x 2 columns]
0.449338    23996
0.526649    18511
0.451151     7576
0.461725     6683
0.452948     5717
            ...  
0.540732        1
0.538556        1
0.520763        1
0.538466        1
0.420964        1
Name: PROBA, Length: 123723, dtype: int64
                                PROBA
1044351  (0.40599999999999997, 0.429]
1117635  (0.40599999999999997, 0.429]
620301   (0.40599999999999997, 0.429]
554085   (0.40599999999999997, 0.429]
299901   (0.40599999999999997, 0.429]
...                               ...
407129                 (0.595, 0.698]
1153801                (0.595, 0.698]
383445                 (0.595,

In [ ]:
#############################################################
#                   MODELE 3 : BAGGING
#############################################################

In [51]:
model_Bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced"), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__max_depth' : list(range(2,20, 1)), 'n_estimators' : [20,50,100],
         'bootstrap':[True], "bootstrap_features": [False]}

grid_Bagging = GridSearchCV(model_Bagging, param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_Bagging = grid_Bagging.fit(train_X_s, train_Y_s)
resultat_grid_Bagging.best_params_


{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'bootstrap': True,
 'bootstrap_features': False,
 'n_estimators': 50}

In [26]:
final_model_BaggingClassifier = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced", criterion="gini", max_depth=2, random_state=0),bootstrap=True, bootstrap_features=False, n_estimators = 50) 
evaluation(final_model_BaggingClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[3289 2042]
 [6708 7821]]
              precision    recall  f1-score   support

           1       0.33      0.62      0.43      5331
           2       0.79      0.54      0.64     14529

    accuracy                           0.56     19860
   macro avg       0.56      0.58      0.54     19860
weighted avg       0.67      0.56      0.58     19860

train set : 
[[12917  8480]
 [26550 31489]]
              precision    recall  f1-score   support

           1       0.33      0.60      0.42     21397
           2       0.79      0.54      0.64     58039

    accuracy                           0.56     79436
   macro avg       0.56      0.57      0.53     79436
weighted avg       0.66      0.56      0.58     79436

GRAPHIQUE NON DEMANDE


In [27]:
vingtiles(model = final_model_BaggingClassifier, data_orig = data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Decision_Tree_Classifier.xlsx")

            IDENT     PROBA
0      0010000001  0.421471
51100  0010376657  0.421471
51106  0010376663  0.421471
51107  0010376664  0.421471
51109  0010376666  0.421471
...           ...       ...
30820  0010351248  0.753873
2297   0010029949  0.753873
33573  0010358016  0.753873
20580  0010336549  0.753873
65100  0010398693  0.753873

[99296 rows x 2 columns]
0.421471    36029
0.530511    24169
0.507569    15519
0.451343    11014
0.656990     1817
0.744772     1804
0.474285     1397
0.477697     1343
0.705378     1167
0.635514      938
0.666091      904
0.753873      883
0.673075      534
0.714480      437
0.644615      377
0.682176      253
0.651598      208
0.712468      113
0.689294      110
0.698395       80
0.660700       72
0.721570       45
0.475478       32
0.423689       11
0.690992        7
0.505350        7
0.676660        6
0.453561        5
0.528293        3
0.685761        3
0.692744        2
0.669624        2
0.701846        2
0.725102        1
0.476504        1
0.700093

ValueError: ignored

In [ ]:
#############################################################
#                   MODELE 4 : EXTRA-TREES
#############################################################

In [20]:
model_ExtraTreesClassifier = ExtraTreesClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'], 'max_depth' : [2,3,4,5],"max_features":[8,12,15,20,25,30] ,'n_estimators' : [20,50,100] }

grid_ExtraTreesClassifier = GridSearchCV(model_ExtraTreesClassifier, param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_ExtraTreesClassifier = grid_ExtraTreesClassifier.fit(train_X_s, train_Y_s)
resultat_grid_ExtraTreesClassifier.best_params_



{'criterion': 'entropy',
 'max_depth': 3,
 'max_features': 30,
 'n_estimators': 50}

In [47]:
final_model_ExtraTreesClassifier = ExtraTreesClassifier( criterion="entropy", max_depth=3, max_features=30, n_estimators=50 , class_weight="balanced", random_state=0) 
evaluation(final_model_ExtraTreesClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[3420 1911]
 [6955 7574]]
              precision    recall  f1-score   support

           1       0.33      0.64      0.44      5331
           2       0.80      0.52      0.63     14529

    accuracy                           0.55     19860
   macro avg       0.56      0.58      0.53     19860
weighted avg       0.67      0.55      0.58     19860

train set : 
[[13414  7983]
 [27659 30380]]
              precision    recall  f1-score   support

           1       0.33      0.63      0.43     21397
           2       0.79      0.52      0.63     58039

    accuracy                           0.55     79436
   macro avg       0.56      0.58      0.53     79436
weighted avg       0.67      0.55      0.58     79436

GRAPHIQUE NON DEMANDE


In [48]:
vingtiles_tableau=vingtiles(model = final_model_ExtraTreesClassifier, data_orig = data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Extra_Trees_Classifier.xlsx")

            IDENT     PROBA
14323  0010261341  0.402797
21707  0010338071  0.402948
61080  0010394340  0.402948
35477  0010360000  0.402948
23635  0010340220  0.402948
...           ...       ...
50923  0010376412  0.718302
75590  0010410331  0.718302
2070   0010027016  0.718302
98902  0010538813  0.718302
15264  0010262550  0.718302

[99296 rows x 2 columns]
0.446976    3932
0.529540    2659
0.528655    1724
0.509002    1678
0.425110    1476
            ... 
0.531967       1
0.521036       1
0.447646       1
0.441156       1
0.472760       1
Name: PROBA, Length: 12253, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            IDENT     PROBA        TRANCHES CLASSE
14323  0010261341  0.402797  (0.402, 0.422]      1
21707  0010338071  0.402948  (0.402, 0.422]      1
61080  0010394340  0.402948  (0.402, 0.422]      1
35477  0010360000  0.402948  (0.402, 0.422]      1
23635  0010340220  0.402948  (0.402, 0.422]      1
...           ...       ...             ...    ...
50923  0010376412  0.718302  (0.596, 0.718]     20
75590  0010410331  0.718302  (0.596, 0.718]     20
2070   0010027016  0.718302  (0.596, 0.718]     20
98902  0010538813  0.718302  (0.596, 0.718]     20
15264  0010262550  0.718302  (0.596, 0.718]     20

[99296 rows x 4 columns]
Top_mon            1     2  TOTAL
TRANCHES                         
(0.402, 0.422]   876  4575   5451
(0.422, 0.427]   767  3714   4481
(0.427, 0.442]   908  4058   4966
(0.442, 0.447]  1346  6148   7494
(0.447, 0.449]   574  1863   2437
(0.449, 0.456]  1082  3887   4969
(0.456, 0.46]   1203  4345   5548
(0.46, 0.468]   1104  3423   4527
(0.468, 0.481]  1173  

In [49]:
# TABLEAU MONTANT
tableau_5_montant(vingtiles_tableau, data_montant_orig, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/montant_Extra_Trees_Classifier.xlsx")

            IDENT     PROBA        TRANCHES CLASSE  SommeDeMONTANT_NUM
0      0010261341  0.402797  (0.402, 0.422]      1                15.0
1      0010338071  0.402948  (0.402, 0.422]      1                30.0
2      0010394340  0.402948  (0.402, 0.422]      1                20.0
3      0010360000  0.402948  (0.402, 0.422]      1                20.0
4      0010340220  0.402948  (0.402, 0.422]      1                20.0
...           ...       ...             ...    ...                 ...
99291  0010376412  0.718302  (0.596, 0.718]     20               200.0
99292  0010410331  0.718302  (0.596, 0.718]     20                50.0
99293  0010027016  0.718302  (0.596, 0.718]     20                40.0
99294  0010538813  0.718302  (0.596, 0.718]     20                50.0
99295  0010262550  0.718302  (0.596, 0.718]     20              1000.0

[99146 rows x 5 columns]
          TRANCHES Moyen_Somme TOTAL
0   (0.402, 0.422]     29.5749  5450
1   (0.422, 0.427]     31.7655  4480
2   (0.427,

In [50]:
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_ExtraTreesClassifier, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/ind_sans_target_Extra_Trees_Classifier.xlsx")

              IDENT     PROBA
280673   0100004652  0.401457
1040582  0360117750  0.402797
116210   0020006751  0.402797
680877   0270013655  0.402797
594529   0220036767  0.402797
...             ...       ...
406052   0130054036  0.718302
660858   0260013636  0.718302
383142   0130031126  0.718302
636562   0250009346  0.718764
895764   0340001932  0.719360

[1092884 rows x 2 columns]
0.446976    49028
0.529540    36402
0.528655    22677
0.425110    20221
0.531098    11654
            ...  
0.588287        1
0.510456        1
0.598328        1
0.597622        1
0.559156        1
Name: PROBA, Length: 37900, dtype: int64
                  PROBA
280673      (0.4, 0.42]
1040582     (0.4, 0.42]
116210      (0.4, 0.42]
680877      (0.4, 0.42]
594529      (0.4, 0.42]
...                 ...
406052   (0.601, 0.719]
660858   (0.601, 0.719]
383142   (0.601, 0.719]
636562   (0.601, 0.719]
895764   (0.601, 0.719]

[1092884 rows x 1 columns]
              IDENT     PROBA        TRANCHES
280673   01

In [ ]:
#############################################################
#                   MODELE 5 : XGBoost (IL FAUT L'AMELIORER CAR GRIDSEARCHCV PREND TROP DE TEMPS)
#############################################################

In [18]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [19]:

model_XGBClassifier = XGBClassifier(learning_rate =0.1, n_estimators=100, gamma=0, subsample=0.8,
                                                  colsample_bytree=0.8, objective= 'binary:logistic', nthread=-1, seed=0)
param = { 'scale_pos_weight': list(np.linspace(0.30, 0.40, 20)), 'max_depth':range(3,10,1), 'min_child_weight':range(1,6,2) } #list(np.linspace(0.10, 0.30, 2))   #'max_depth':range(3,10,2), 'min_child_weight':range(1,6,2),

grid_XGBClassifier = GridSearchCV(estimator = model_XGBClassifier, param_grid = param, scoring='recall', cv=5, n_jobs=-1)
resultat_grid_XGBClassifier = grid_XGBClassifier.fit(train_X_s, train_Y_s)
resultat_grid_XGBClassifier.best_params_

KeyboardInterrupt: ignored

In [ ]:
final_model_XGBClassifier = XGBClassifier(learning_rate =0.1, n_estimators=100, max_depth=3, min_child_weight=1, gamma=0, subsample=0.8,
                                                  colsample_bytree=0.8, objective= 'binary:logistic', nthread=2, scale_pos_weight=0.34, seed=0)
evaluation(final_model_XGBClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[3313 2018]
 [6270 8259]]
              precision    recall  f1-score   support

           1       0.35      0.62      0.44      5331
           2       0.80      0.57      0.67     14529

    accuracy                           0.58     19860
   macro avg       0.57      0.59      0.56     19860
weighted avg       0.68      0.58      0.61     19860

train set : 
[[13348  8049]
 [24432 33607]]
              precision    recall  f1-score   support

           1       0.35      0.62      0.45     21397
           2       0.81      0.58      0.67     58039

    accuracy                           0.59     79436
   macro avg       0.58      0.60      0.56     79436
weighted avg       0.68      0.59      0.61     79436

GRAPHIQUE NON DEMANDE


In [ ]:
vingtiles_xgb(model = final_model_XGBClassifier, data_orig = data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_XGBoost.xlsx")

                 IDENT     PROBA
66003  660040010399617  0.227716
67823  678240010401567  0.227716
78844  788450010414774  0.227716
29727  297280010349859  0.232173
68075  680760010401859  0.236831
...                ...       ...
48128  481290010373476  0.885968
83091  830920010502100  0.885968
5615    56160010208685  0.885968
83536  835370010503920  0.886497
60221  602220010392936  0.888440

[99296 rows x 2 columns]
0.546209    385
0.495340    256
0.403445    226
0.550064    213
0.431736    197
           ... 
0.700457      1
0.412724      1
0.475223      1
0.412721      1
0.437500      1
Name: PROBA, Length: 58312, dtype: int64
                PROBA
66003  (0.227, 0.361]
67823  (0.227, 0.361]
78844  (0.227, 0.361]
29727  (0.227, 0.361]
68075  (0.227, 0.361]
...               ...
48128  (0.717, 0.888]
83091  (0.717, 0.888]
5615   (0.717, 0.888]
83536  (0.717, 0.888]
60221  (0.717, 0.888]

[99296 rows x 1 columns]
Top_mon            1     2  TOTAL
TRANCHES                         
(0.

In [ ]:
###################################################################################################################@
###################################################################################################################@
#                                   DECISION TREE CLASSIFIER  ----  AVEC VARIABLES QUALITATIVES
###################################################################################################################@
###################################################################################################################@

In [ ]:
####################################################################################
####################################################################################
#                   DECISION TREE
####################################################################################
####################################################################################

In [ ]:

model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, class_weight = "balanced" )
param = { 'max_depth' :list(range(2,20)), 'criterion': ["gini", "entropy"], 'max_features' : list(range(3,10,1)),
         'min_samples_split' : [0.05, 0.1, 0.4, 0.6, 0.8, 1] }

grid_DecisionTreeClassifier = GridSearchCV(model_DecisionTreeClassifier, param_grid= param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_DecisionTreeClassifier = grid_DecisionTreeClassifier.fit(train_X_s, train_Y_s)
resultat_grid_DecisionTreeClassifier.best_params_


{'criterion': 'gini',
 'max_depth': 2,
 'max_features': 5,
 'min_samples_split': 0.05}

In [ ]:
final_model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, criterion="gini", class_weight = "balanced", max_depth = 2, max_features=13, min_samples_split=0.05 )
evaluation(final_model_DecisionTreeClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 5180   151]
 [13889   640]]
              precision    recall  f1-score   support

           1       0.27      0.97      0.42      5331
           2       0.81      0.04      0.08     14529

    accuracy                           0.29     19860
   macro avg       0.54      0.51      0.25     19860
weighted avg       0.66      0.29      0.18     19860

train set : 
[[20819   578]
 [55499  2540]]
              precision    recall  f1-score   support

           1       0.27      0.97      0.43     21397
           2       0.81      0.04      0.08     58039

    accuracy                           0.29     79436
   macro avg       0.54      0.51      0.25     79436
weighted avg       0.67      0.29      0.18     79436

GRAPHIQUE NON DEMANDE


In [ ]:
# TRAIN SET TEST SET
vingtiles(model = final_model_DecisionTreeClassifier, data_orig = data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Decision_Tree_Classifier.xlsx")

                 IDENT     PROBA
56970  569710010384479  0.381666
9253    92540010256064  0.381666
52518  525190010378240  0.381666
60182  601830010392815  0.381666
22850  228510010339375  0.381666
...                ...       ...
29519  295200010349528  0.553444
94344  943450010532481  0.553444
69420  694210010403242  0.553444
44966  449670010370130  0.553444
49402  494030010374795  0.553444

[99296 rows x 2 columns]
0.501674    90709
0.553444     4678
0.381666     3909
Name: PROBA, dtype: int64


ValueError: ignored

In [ ]:
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_DecisionTreeClassifier, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/ind_sans_target_Decision_Tree_Classifier.xlsx")